#### Suppose each individual in a population experiences exponential lifespans. Then if an individual $i$ has rate parameter $\rho_i$, their survival probability until age $a$ is $s(a) = e^{-\rho_i a}$.  In discrete time, if an individual has death probability $d$, then their survival probability until age $a$ is $s(a) = (1-d)^a$. 

#### If everybody gets the same $\rho$, the population survival function is the same as the individual's. But if $\rho\sim\mathcal{D}$ for some distribution $\mathcal{D}$ with pdf $g(\rho)$, the population's survival function is different. For instance, if $\rho = \rho_j$ w.p. $p_j$ for $j = 1, \ldots, n$ then the population's suvival function is $s(a) = \sum p_j e^{-\rho_j a}$ (in continuous time - discrete time is analogous). If $\rho$ is continuously distributed, then the population's survival function is $s(a) = \int_0^\infty g(\rho) e^{-\rho a} d \rho$. 

#### Holding $\mathbb{E}[\rho]$ constant, how does distribution of $\rho$ impact survival probability and fitness? Naturally, different distributions result in different survival functions -- I feel like different fitnesses as well, though that one is a little more difficult to justify. But if we have different fitnesses, it seems like we could try to optimize fitness over the space of all distributions with fixed expected rate parameter.  

In [1]:
import matplotlib.pyplot as plt

In [3]:
def Leslie(fecundity, survival):
    n = len(fecundity)
    survival = survival[:n-1]
    L = zero_matrix(QQ, n)
    for i, b in enumerate(fecundity):
        L[0, i] = b
    for i, s in enumerate(survival):
        L[i + 1, i] = s
    return(L)

def is_essentially_real(x):
    if x.imag() == 0:
        return(True)
    else:
        return(False)
    
#Use of "if is_essentiallY_real(e)" rather than "if e in RR" is required since some computational errors
#    seem to come up in the eigenvalue computation, giving us things like x + 0.?e-80*I.

def get_leading_eigenvalue(L):
    evals = L.eigenvalues()
    moduli = [e.n() for e in evals if is_essentially_real(e)]
    moduli = [e for e in moduli if e >= 0]
    r = max(moduli)
    return(r)

def normalize(vec):
    tot = sum(vec)
    vec = vec/tot
    return(vec)

def get_leading_evec(L):
    r = get_leading_eigenvalue(L)
    evecs = L.eigenvectors_right()
    arrs = [e[0].n() for e in evecs]
    i = arrs.index(r)
    leading_evec = evecs[i][1][0]
    leading_evec = normalize(leading_evec).n()
    return(leading_evec)

#### Let $\gamma$s be distribution of death probability $d$. Let $\gamma_1 := \delta_d(\cdot)$ and $\gamma_2:=\frac{1}{2}\delta_{d - \varepsilon} + \frac{1}{2}\delta_{d + \varepsilon}$. In the first case, the entire population gets the same death probability $d$ whereas in the second, an individual at birth is assigned either $d - \varepsilon$ or $d + \varepsilon$ with equal probability. One can verify that $\mathbb{E}[d_i] = d$ in each case. 

In [30]:
def s1(a, d = 0.05):
    """
    Population and individual level probability of surviving to age a given a.s. death rate d.
    """
    return (1 - d)^(a)

def s2(a, d = 0.05, epsilon = 0.02):
    if epsilon > d:
        print('Make sure epsilon < d')
        return None
    """
    Population level probability of surviving to age a. At birth, assigned death 
    prob d - epsilon w.p. 1/2, d + epsilon otherwise. 
    """
    return 1/2*(1 - d + epsilon)^a + 1/2*(1 - d - epsilon)^a

def instantaneous(surv_func, num = 10):
    return [surv_func(a+1)/surv_func(a) for a in range(num)]

In [31]:
instantaneous(s1, 10)

[0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000,
 0.950000000000000]

In [32]:
instantaneous(s2, 10)

[0.950000000000000,
 0.950421052631579,
 0.950841732196256,
 0.951261666948160,
 0.951680487773549,
 0.952097829482185,
 0.952513332069434,
 0.952926641939914,
 0.953337413084017,
 0.953745308199188]

#### I'm finding that it is universally better to be a $\gamma_2$ individual. Your instantaneous survival probability starts of the same as the $\gamma_1$ types, and then it just gets better and better. Can I find a case where it is better to have a.s. $d$?

In [36]:
def s_unbalanced(a, d = 0.05, epsilon = 0.02):
    """
    Population level survival probability to age a. Born with death rate d - 9*epsilon/10 w.p. 1/10 and
    d + epsilon/10 w.p. 9/10. 
    """
    return 9/10*(1 - d + epsilon/10)^a + 1/10*(1 - d - 9*epsilon/10)^a


In [38]:
instantaneous(s_unbalanced)

[0.950000000000000,
 0.950037894736842,
 0.950075148248934,
 0.950111768489787,
 0.950147763425439,
 0.950183141029278,
 0.950217909277070,
 0.950252076142161,
 0.950285649590875,
 0.950318637578088]

#### Still finding the same result. I rationalise it as follows: the first round, exactly $d$ die, since that is the average probability of death. But more of those who die are the ones who have higher death probability. Essentially, what we are doing is sacrificing some in order to have higher survival probability among those who make it through. The long run instantaneous survival of the population is the instantaneous survival of the type with lowest $d$.